In [1]:
import pandas as pd
from litecoder.usa import USCityIndex, USStateIndex
idx_city = USCityIndex()
idx_state = USStateIndex()

idx_city.load()
idx_state.load()

/Users/hjian42/opt/anaconda3/lib/python3.7/site-packages/litecoder/models/wof_locality.py:24: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  'litecoder', 'data/city-alt-names.yml'


In [2]:
def extract_city_state(L):
    """
    returns city, state, country
    """
    if L:
        L = str(L)
    
        if L.lower() in set(["united states", "the united states", "u.s.", "us", "usa", "u.s.a", "u.s.a."]):
            return "", "", "US"

        r = idx_city[L]
        if r:
            return r[0]['name'], r[0]['name_a1'], r[0]['country_iso']
        r = idx_state[L]
        if r:
            return "", r[0]['name'], r[0]['country_iso']
    
    return "", "", ""

extract_city_state("boston ma")

('Boston', 'Massachusetts', 'US')

In [3]:
def extract_location_from_user(user):
    user = ast.literal_eval(user) if user==user else ""
    location = user['location'] if "location" in user else ""
    return extract_city_state(location)

## all the users by week

In [12]:
from glob import glob
import ast
from pandarallel import pandarallel

pandarallel.initialize()

for filename in glob("./minimal/*.csv"):
    print(filename)
    df = pd.read_csv(filename)
    df['triple_loc'] = df.location.parallel_apply(extract_city_state)
    df['city'] = df.triple_loc.parallel_apply(lambda x: x[0])
    df['state'] = df.triple_loc.parallel_apply(lambda x: x[1])
    df['country'] = df.triple_loc.parallel_apply(lambda x: x[2])
    df = df.drop(["triple_loc", 'Unnamed: 0'], axis=1)
    df = df[df.state == ""]
#     print(df)
    out_filename = filename.replace("minimal", "minimal-output")
    df.to_csv(out_filename, index=False)
#     break

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
./minimal/6_4.csv
./minimal/12_18.csv
./minimal/14_9.csv
./minimal/4_6.csv
./minimal/October_3.csv
./minimal/October_2.csv
./minimal/April_4.csv
./minimal/4_7.csv
./minimal/12_19.csv
./minimal/14_8.csv
./minimal/6_5.csv
./minimal/6_7.csv
./minimal/8_8.csv
./minimal/4_5.csv
./minimal/7_31.csv
./minimal/October_0.csv
./minimal/October_1.csv
./minimal/4_4.csv
./minimal/8_9.csv
./minimal/6_6.csv
./minimal/6_2.csv
./minimal/4_0.csv
./minimal/April_3.csv
./minimal/dec_19.csv
./minimal/5_19.csv
./minimal/12_9.csv
./minimal/12_8.csv
./minimal/5_18.csv
./minimal/dec_18.csv
./minimal/April_2.csv
./minimal/October_4.csv
./minimal/4_1.csv
./minimal/6_3.csv
./minimal/6_1.csv
./minimal/4_3.csv
./minimal/April_0.csv
./minimal/10_8.csv
./minimal/10_9.csv
./minimal/April_1.csv
./minimal/4_2.csv
./minimal/6_0.csv
./minimal/3_3.csv
./min

In [13]:
df

,week,location,city,state,country
0,71,⬅️ 2m ➡️ Sans social bubble,,,
2,72,Jakarta Capital Region,,,
3,72,"Kigali, Rwanda",,,
4,72,Australia,,,
5,72,NaN,,,
...,...,...,...,...,...
1498016,73,United States,,,US
1498017,73,NaN,,,
1498018,75,Northen Hemisphere,,,
1498019,73,Oxford,,,


## drug users

In [4]:
from glob import glob
import ast

for filename in glob("./*.csv"):
    print(filename)
    df = pd.read_csv(filename)
#     df = df.head(1000)
    df['triple_loc'] = df.user.apply(extract_location_from_user)
#     df['triple_loc'] = df.user.apply(lambda x: extract_city_state(ast.literal_eval(x)['location']))
    df['city'] = df.triple_loc.apply(lambda x: x[0])
    df['state'] = df.triple_loc.apply(lambda x: x[1])
    df['country'] = df.triple_loc.apply(lambda x: x[2])
    df = df.drop(["triple_loc"], axis=1)
    df.to_csv("./output/new_{}".format(filename[2:]))
#     break
# df_12months['triple_loc'] = df_12months.location.apply(extract_city_state)
#     break

./hcq_all.csv
./molnupiravir_all.csv


/Users/hjian42/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./ivermectin_all.csv


/Users/hjian42/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./remdesivir_all.csv


In [56]:
for filename in glob("./output/*.csv"):
    print(filename)
    df = pd.read_csv(filename)
    remain_num, total_num = df[df.country == "US"].shape[0], df.shape[0]
    print("before de-dup:", remain_num, total_num, remain_num/total_num)
    df['name_tweet'] = df.screen_name + df.full_text
    df = df.drop_duplicates(subset='name_tweet', keep='first')
    remain_num, total_num = df[df.country == "US"].shape[0], df.shape[0]
    print("after de-dup:", remain_num, total_num, remain_num/total_num)


./output/new_molnupiravir_all.csv


/Users/hjian42/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


before de-dup: 17862 66149 0.270026757774116
after de-dup: 17536 65034 0.26964357105514036
./output/new_hcq_all.csv


/Users/hjian42/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


before de-dup: 184856 646553 0.285910049137503
after de-dup: 182310 613640 0.2970960172087869
./output/new_remdesivir_all.csv
before de-dup: 45598 195217 0.23357596930595184
after de-dup: 44034 190080 0.23166035353535352
./output/new_ivermectin_all.csv


/Users/hjian42/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2,5,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


before de-dup: 153486 682568 0.22486550790543947
after de-dup: 151443 669080 0.2263451306271298


In [55]:
df.shape[0]

669080